## Select electrodes for each subject

In [20]:
import numpy as np
from brainpipe.system import study
from os import path

### Remove bad trials
    Before concatenation in Good vs Bad

In [18]:
path = r'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/OE_Matrices_NoArt/TS_E_all_by_odor_th40_art400_30_250_5s_expi/VACJ/'
# for LEFC :: 3 trials to remove
x1 = np.load(path+'E1/correct_ts_16_by_odor_trigs.npy')
x2 = np.load(path+'E1/correct_ts_15_by_odor_trigs.npy')
x3 = np.load(path+'E2/correct_ts_12_by_odor_trigs.npy')
# x4 = np.load(path+'E2/correct_ts_5_by_odor_trigs.npy')
print('shapes', x1.shape, x2.shape, x3.shape)
new_x1 = np.delete(x1,[1], axis=0)
new_x2 = np.delete(x2,[0], axis=0)
new_x3 = np.delete(x3,[3], axis=0)
# new_x4 = np.delete(x4,[], axis=0)
print('new shapes', new_x1.shape, new_x2.shape, new_x3.shape)
np.save(path+'E1/correct_ts_16_by_odor_trigs.npy',new_x1)
np.save(path+'E1/correct_ts_15_by_odor_trigs.npy',new_x2)
np.save(path+'E2/correct_ts_12_by_odor_trigs.npy',new_x3)
# np.save(path+'E2/correct_ts_5_by_odor_trigs.npy',new_x4)

shapes (4, 152, 2816) (4, 152, 2816) (4, 152, 2816)
new shapes (3, 152, 2816) (3, 152, 2816) (3, 152, 2816)


### Remove electrodes in WM, LCR and skull
    For all subjects

In [9]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
badname = np.load(pathfiles+'PIRJ_odor_bad_bipo_new.npz')
print(badname.files)
mat = badname['x']
print(mat.shape)

-> Olfacto loaded
['channel', 'label', 'xyz', 'x', 'sf']
(71, 2816, 41)


In [21]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
#subjects = ['CHAF','VACJ','SEMC', 'LEFC','MICP','FERJ','PIRJ'] #'CHAF','VACJ','SEMC', 'LEFC','MICP', 
subjects = ['VACJ']
phases = ['odor']
for su in subjects:
    for phase in phases:
        print(su)
        #load all subjects information
        badname = np.load(pathfiles+su+'_odor_bad_bipo.npz')
        goodname = np.load(pathfiles+su+'_odor_good_bipo.npz')
        bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
        good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
        sf = goodname['sf']
        print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
        print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)
        print('labels',good_label)
        
        #create a list of all electrodes to remove
        rois_to_rej = ['WM','skull','out','LCR']
        elecs_rej = []
        for elec in range(bad_label.shape[0]):
            if any(x in bad_label[elec] for x in rois_to_rej):
                elecs_rej.append(elec)
            else: continue

        #Remove all bad electrodes from the database
        new_bad, new_bad_label = np.delete(bad,elecs_rej, axis=0), np.delete(bad_label,elecs_rej, axis=0)
        new_bad_channel, new_bad_xyz = np.delete(bad_channel,elecs_rej, axis=0), np.delete(bad_xyz,elecs_rej, axis=0)
        new_good, new_good_label = np.delete(good,elecs_rej, axis=0), np.delete(good_label,elecs_rej, axis=0)
        new_good_channel, new_good_xyz = np.delete(good_channel,elecs_rej, axis=0), np.delete(good_xyz,elecs_rej, axis=0)
        print (su,phase,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
              'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)

        #Save all new information
        mat_good = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
        mat_bad = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
        file_good = su+'_'+phase+'_good_bipo_new.npz'
        file_bad = su+'_'+phase+'_bad_bipo_new.npz'
        np.savez(pathfiles+file_good,**mat_good)
        np.savez(pathfiles+file_bad,**mat_bad)
        del elecs_rej, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
        del new_good_label, new_good_channel, new_good_xyz

-> Olfacto loaded
VACJ
bad shape:  (139, 2816, 14) (139,) (139,) (139, 3)
good shape:  (139, 2816, 15) (139,) (139,) (139, 3)
labels ['mHC-Ent' 'mHC-PHG&mHC-Ent' 'mHC-PHG-FuG&mHC-PHG' 'WM-FuG&mHC-PHG-FuG'
 'MTG&WM-FuG' 'MTG' 'MTG' 'MTG' 'MTG' 'MTG' 'MTG' 'mHC' 'mHC' 'mHC' 'mHC'
 'mHC' 'WM-mHC&mHC' 'WM-MTG&WM-mHC' 'WM-MTG' 'MTG&WM-MTG' 'MTG' 'MTG'
 'Amg-PRC&Amg' 'Amg-PRC' 'PRC&Amg-PRC' 'WM-PRC&PRC' 'WM-Ppo&WM-PRC'
 'WM-Ppo' 'MTG&WM-Ppo' 'MTG' 'MTG' 'MTG' 'MTG' 'IFGOp-IG'
 'IFGOp-IFGTr&IFGOp-IG' 'IFGOp-IFGTr' 'IFGOp-IFGTr' 'WM-IFGOp&IFGOp-IFGTr'
 'WM-IFGOp' 'IFGOp-IFGTr&WM-IFGOp' 'IFGOp-IFGTr' 'LCR&IFGOp-IFGTr' 'SRoG'
 'WM-SRoG&SRoG' 'WM-IFPG&WM-SRoG' 'WM-IFPG' 'WM-IFPG' 'WM-IFPG'
 'WM-MFG&WM-IFPG' 'MFG&WM-MFG' 'MFG' 'MFG' 'MFG-LCR&MFG'
 'WM-ITG-MTG&WM-PPo' 'MTG&WM-ITG-MTG' 'MTG' 'LCR-MTG&MTG' 'LCR&LCR-MTG'
 'LCR-skull&LCR' 'skull&LCR-skull' 'pPirtT&pPirT-Ent' 'WM-pPirT&pPirtT'
 'WM-ITG&WM-pPirT' 'WM-MTG&WM-ITG' 'WM-MTG' 'MTG&WM-MTG' 'MTG' 'MTG'
 'LCR-MTG&MTG' 'pgACC' 'WM-pgACC&pgACC' 'W

### Select some electrodes

In [2]:
import numpy as np
from brainpipe.system import study
from os import path
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
su = 'LEFC'
################ LOAD FILES WITH ALL ELECTRODES ###########################################
badname = np.load(pathfiles+su+'_odor_bad_bipo_new.npz')
goodname = np.load(pathfiles+su+'_odor_good_bipo_new.npz')
bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
sf = goodname['sf']
print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)

############# CREATE A LIST OF ELECTRODES TO REMOVE #######################################
all_elec = ['a','b','d','j','q','s','t']
# Find the elecs id to remove them
elecs_id = []
for elec in range(bad_label.shape[0]):
    if any(x in bad_channel[elec] for x in all_elec):
        elecs_id.append(elec)
    else: continue
print(elecs_id)
################ REMOVE ALL ELECS INFORMATION FROM THE DATABASE #######################################
new_bad, new_bad_label = np.take(bad,elecs_id, axis=0), np.take(bad_label,elecs_id, axis=0)
new_bad_channel, new_bad_xyz = np.take(bad_channel,elecs_id, axis=0), np.take(bad_xyz,elecs_id, axis=0)
new_good, new_good_label = np.take(good,elecs_id, axis=0), np.take(good_label,elecs_id, axis=0)
new_good_channel, new_good_xyz = np.take(good_channel,elecs_id, axis=0), np.take(good_xyz,elecs_id, axis=0)
print (su,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
      'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)
print(new_bad_channel)
############### SAVE ELECTRODES TO ANALYZE ##################################################
mat_good = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
mat_bad = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
file_good = su+'_odor_good_bipo_MTL_PrG.npz'
file_bad = su+'_odor_bad_bipo_MTL_PrG.npz'
np.savez(pathfiles+file_good,**mat_good)
np.savez(pathfiles+file_bad,**mat_bad)
del elecs_id, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
del new_good_label, new_good_channel, new_good_xyz

-> Olfacto loaded
bad shape:  (152, 2816, 23) (152,) (152,) (152, 3)
good shape:  (152, 2816, 35) (152,) (152,) (152, 3)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 36, 37, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
LEFC bad (53, 2816, 23) (53,) (53,) (53, 3) good (53, 2816, 35) (53,) (53,) (53, 3)
['a2-a1' 'a6-a5' 'a7-a6' 'a8-a7' 'a9-a8' 'a10-a9' 'a11-a10' 'b2-b1' 'b3-b2'
 'b4-b3' 'b9-b8' 'b10-b9' 'b11-b10' 'd2-d1' 'd3-d2' 'd4-d3' 'd5-d4' 'd6-d5'
 'd7-d6' 'd8-d7' 'd9-d8' 'd10-d9' 'd11-d10' 'j2-j1' 'j3-j2' 'q2-q1' 'q3-q2'
 'q4-q3' 'q5-q4' 'q6-q5' 'q7-q6' 'q8-q7' 'q9-q8' 'q10-q9' 'q11-q10' 's2-s1'
 's3-s2' 's4-s3' 's5-s4' 's6-s5' 's7-s6' 's8-s7' 's9-s8' 's10-s9' 's11-s10'
 't2-t1' 't3-t2' 't4-t3' 't5-t4' 't6-t5' 't7-t6' 't8-t7' 't9-t8']


In [2]:
import numpy as np
from brainpipe.system import study
from os import path
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
su = 'LEFC'

################ LOAD FILES WITH ALL ELECTRODES ###########################################
badname = np.load(pathfiles+su+'_odor_bad_bipo_new.npz')
goodname = np.load(pathfiles+su+'_odor_good_bipo_new.npz')
bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
sf = goodname['sf']
print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)

############# CREATE A LIST OF ELECTRODES TO REMOVE #######################################
all_elec = ['g4','g5','g6','g10','g11','j1','j2','j3']
w = ['w{0}'.format(s) for s in range(9,13)]
all_elec_remove = w+all_elec
# Find the elecs id to remove them
elecs_id = []
for elec in range(bad_label.shape[0]):
    if any(x in bad_channel[elec] for x in all_elec_remove):
        elecs_id.append(elec)
    else: continue
print(elecs_id)

################ REMOVE ALL ELECS INFORMATION FROM THE DATABASE #######################################
new_bad, new_bad_label = np.delete(bad,elecs_id, axis=0), np.delete(bad_label,elecs_id, axis=0)
new_bad_channel, new_bad_xyz = np.delete(bad_channel,elecs_id, axis=0), np.delete(bad_xyz,elecs_id, axis=0)
new_good, new_good_label = np.delete(good,elecs_id, axis=0), np.delete(good_label,elecs_id, axis=0)
new_good_channel, new_good_xyz = np.delete(good_channel,elecs_id, axis=0), np.delete(good_xyz,elecs_id, axis=0)
print (su,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
      'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)
print(new_bad_channel)
############### SAVE ELECTRODES TO ANALYZE ##################################################
mat_good = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
mat_bad = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
file_good = su+'_odor_good_bipo_sel.npz'
file_bad = su+'_odor_bad_bipo_sel.npz'
np.savez(pathfiles+file_good,**mat_good)
np.savez(pathfiles+file_bad,**mat_bad)
del elecs_id, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
del new_good_label, new_good_channel, new_good_xyz

-> Olfacto loaded
bad shape:  (152, 2816, 23) (152,) (152,) (152, 3)
good shape:  (152, 2816, 35) (152,) (152,) (152, 3)
[25, 26, 27, 28, 31, 32, 33, 36, 37, 127, 128, 129, 130]
LEFC bad (139, 2816, 23) (139,) (139,) (139, 3) good (139, 2816, 35) (139,) (139,) (139, 3)
['a2-a1' 'a6-a5' 'a7-a6' 'a8-a7' 'a9-a8' 'a10-a9' 'a11-a10' 'b2-b1' 'b3-b2'
 'b4-b3' 'b9-b8' 'b10-b9' 'b11-b10' 'd2-d1' 'd3-d2' 'd4-d3' 'd5-d4' 'd6-d5'
 'd7-d6' 'd8-d7' 'd9-d8' 'd10-d9' 'd11-d10' 'g2-g1' 'g3-g2' 'g8-g7' 'g9-g8'
 'g13-g12' 'g14-g13' 'k2-k1' 'k3-k2' 'k4-k3' 'k8-k7' 'k9-k8' 'k10-k9'
 'k11-k10' 'k12-k11' 'l2-l1' 'l3-l2' 'l4-l3' 'l5-l4' 'l6-l5' 'l7-l6'
 'l8-l7' 'l9-l8' 'l10-l9' 'l11-l10' 'm2-m1' 'm3-m2' 'm10-m9' 'm11-m10'
 'm12-m11' 'm13-m12' 'm14-m13' 'o2-o1' 'o3-o2' 'o4-o3' 'o5-o4' 'o6-o5'
 'o7-o6' 'o8-o7' 'o9-o8' 'o10-o9' 'o11-o10' 'q2-q1' 'q3-q2' 'q4-q3' 'q5-q4'
 'q6-q5' 'q7-q6' 'q8-q7' 'q9-q8' 'q10-q9' 'q11-q10' 'r2-r1' 'r8-r7' 'r9-r8'
 'r10-r9' 'r11-r10' 'r12-r11' 'r13-r12' 'r14-r13' 's2-s1' 's3-s2' 's4

In [22]:
import numpy as np
from brainpipe.system import study
from os import path
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
su = 'VACJ'

################ LOAD FILES WITH ALL ELECTRODES ###########################################
badname = np.load(pathfiles+su+'_odor_bad_bipo_new.npz')
goodname = np.load(pathfiles+su+'_odor_good_bipo_new.npz')
bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
sf = goodname['sf']
print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)

############# CREATE A LIST OF ELECTRODES TO REMOVE #######################################
# all_elec = ['j1','j2','j3']
# w = ['w{0}'.format(s) for s in range(9,13)]
# all_elec_remove = w+all_elec
 # Find the elecs id to remove them
#elecs_id = []
#for elec in range(bad_label.shape[0]):
#    if any(x in bad_channel[elec] for x in ['j8','j7']):
#        elecs_id.append(elec)
#    else: continue
#print(elecs_id)

a,b,c = list(range(0,3)),list(range(9,14)),list(range(16,19))
elecs_id = a+b+c
print(elecs_id)

################ REMOVE ALL ELECS INFORMATION FROM THE DATABASE #######################################
new_bad, new_bad_label = np.delete(bad,elecs_id, axis=0), np.delete(bad_label,elecs_id, axis=0)
new_bad_channel, new_bad_xyz = np.delete(bad_channel,elecs_id, axis=0), np.delete(bad_xyz,elecs_id, axis=0)
new_good, new_good_label = np.delete(good,elecs_id, axis=0), np.delete(good_label,elecs_id, axis=0)
new_good_channel, new_good_xyz = np.delete(good_channel,elecs_id, axis=0), np.delete(good_xyz,elecs_id, axis=0)
print (su,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
      'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)
print(new_bad_channel)
############### SAVE ELECTRODES TO ANALYZE ##################################################
mat_bad = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
mat_good = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
file_good = su+'_odor_good_bipo_sel.npz'
file_bad = su+'_odor_bad_bipo_sel.npz'
np.savez(pathfiles+file_good,**mat_good)
np.savez(pathfiles+file_bad,**mat_bad)
del elecs_id, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
del new_good_label, new_good_channel, new_good_xyz

-> Olfacto loaded
bad shape:  (84, 2816, 15) (84,) (84,) (84, 3)
good shape:  (84, 2816, 14) (84,) (84,) (84, 3)
[0, 1, 2, 9, 10, 11, 12, 13, 16, 17, 18]
VACJ bad (73, 2816, 15) (73,) (73,) (73, 3) good (73, 2816, 14) (73,) (73,) (73, 3)
['b7-b6' 'b8-b7' 'b9-b8' 'b10-b9' 'b11-b10' 'b12-b11' "b'11-b'10"
 "b'12-b'11" "d'9-d'8" "d'10-d'9" "d'11-d'10" "d'12-d'11" "e'2-e'1"
 "e'3-e'2" "e'4-e'3" "e'5-e'4" "e'9-e'8" 'f2-f1' 'f10-f9' 'f11-f10' 'j4-j3'
 "j'2-j'1" "j'8-j'7" "j'9-j'8" 'k2-k1' 'k8-k7' 'k9-k8' 'k10-k9' 'k11-k10'
 'k12-k11' 'k13-k12' "k'2-k'1" "k'12-k'11" "k'13-k'12" "k'14-k'13" 'l2-l1'
 'l3-l2' 'l4-l3' 'l5-l4' 'l6-l5' 'l7-l6' 'l8-l7' 'l9-l8' 'l10-l9' 'l11-l10'
 'o2-o1' 'o3-o2' 'o4-o3' 'o5-o4' 'o6-o5' 'o7-o6' 'o8-o7' 'o9-o8' 'o10-o9'
 'o11-o10' "o'2-o'1" "o'3-o'2" "o'6-o'5" "o'7-o'6" "o'8-o'7" "o'9-o'8"
 "o'10-o'9" "o'11-o'10" "o'12-o'11" 't2-t1' 't3-t2' 't4-t3' 't5-t4' 't6-t5'
 't7-t6' 't8-t7' 't9-t8' 't10-t9']


### Select electrodes in MTL , Frontal OrG and ACC for all subjects

In [ ]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
subjects = ['CHAF','VACJ','SEMC','LEFC','MICP','PIRJ','FERJ'] 

for su in subjects:
    #load all subjects information
    badname = np.load(pathfiles+su+'_odor_bad_bipo_new.npz')
    goodname = np.load(pathfiles+su+'_odor_good_bipo_new.npz')
    bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
    good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
    sf = goodname['sf']
    print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
    print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)
    #print('labels', bad_label)

    #create a list of all electrodes to keep
    elecs_sel = []
    rois = ['HC','PHC','Pir','Amg','ACC','OrG','MTG','ITG','STG']
    for elec in range(bad_label.shape[0]):
        if any(x in bad_label[elec] for x in rois):
            elecs_sel.append(elec)
        else: continue

    #Select electrodes from the database
    new_bad, new_bad_label = np.take(bad,elecs_sel, axis=0), np.take(bad_label,elecs_sel, axis=0)
    new_bad_channel, new_bad_xyz = np.take(bad_channel,elecs_sel, axis=0), np.take(bad_xyz,elecs_sel, axis=0)
    new_good, new_good_label = np.take(good,elecs_sel, axis=0), np.take(good_label,elecs_sel, axis=0)
    new_good_channel, new_good_xyz = np.take(good_channel,elecs_sel, axis=0), np.take(good_xyz,elecs_sel, axis=0)
    print (su,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
          'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)

    #Save all new information
    mat_good = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
    mat_bad = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
    file_good = su+'_odor_good_bipo_new.npz'
    file_bad = su+'_odor_bad_bipo_new.npz'
    np.savez(pathfiles+file_good,**mat_good)
    np.savez(pathfiles+file_bad,**mat_bad)
    del elecs_sel, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
    del new_good_label, new_good_channel, new_good_xyz
